In [1]:
from tqdm.auto import tqdm
import glob
import shutil

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [2]:
import requests
import os

# URL and filename for the Python file
python_file_url = 'https://raw.githubusercontent.com/ajinkya-kulkarni/PyBlendPatches/main/modules.py'
python_filename = 'modules.py'

# Remove the existing file if it exists
if os.path.exists(python_filename):
    os.remove(python_filename)

# Download and save the Python file
response = requests.get(python_file_url)
response.raise_for_status()

with open(python_filename, 'wb') as file:
    file.write(response.content)

In [3]:
model_path = os.path.join('Pretrained_Model', 'Cellpose_Pretrained_Model.379168')

In [4]:
if os.path.exists('AlgorithmPredictions'):
    shutil.rmtree('AlgorithmPredictions')
os.mkdir('AlgorithmPredictions')

In [5]:
import sys
# Configuration to avoid generating __pycache__ and limit traceback for cleaner exceptions
sys.dont_write_bytecode = True
sys.tracebacklimit = 0

from modules import *

In [6]:
lbl_cmap = random_label_cmap()

In [7]:
from cellpose import models

def predict_mask_from_image_cellpose(normalized_img_patch, gpu_usage = False):

    model = models.CellposeModel(gpu = gpu_usage, pretrained_model = model_path)
    
    channels = [[0, 0]]

    """
    Predicts a segmentation mask from an image patch using a given model.
    Args:
    normalized_img_patch (numpy array): The normalized image patch for prediction.
    model (object): The segmentation model used for predicting the mask.
    channels (list): A list of channels to be used in the prediction.

    Returns:
    numpy array: The predicted segmentation mask.
    """

    mask, flow, style = model.eval(normalized_img_patch, diameter=None, channels=channels)

    # Check if the mask is empty (all zeros)
    if np.any(mask):
        # Return the predicted mask if it's not empty
        return mask.astype('uint16')
    else:
        # Return an array of zeros if the mask is empty
        return np.zeros(normalized_img_patch.shape, dtype=np.uint16)

In [8]:
folder_path = os.path.join('/home', 'ajinkya', 'Desktop', 'PyOrganoidAnalysis', 'Dataset', 'Test')
data_dir = os.path.join(folder_path, 'Images', '*.tif')

predict_images_path = sorted([f for f in glob.glob(data_dir)])

In [9]:
window_size = 800
overlap = int(0.5 * window_size)

In [10]:
images_array = []
labels_array = []

for image_path in tqdm(predict_images_path, desc='Reading images and predicting', leave = True):
    
    normalized_img = read_image_as_grayscale_then_MinMax_normalize(image_path)
        
    ###########################################################################################

    patches, window_coords = patchify(normalized_img, window_size, overlap)
    
    ###########################################################################################
    
    predicted_labels = []

    for patch in tqdm(patches, desc = 'Predicting on patches', leave = False):
    
        label = predict_mask_from_image_cellpose(patch, gpu_usage = True)
    
        smoothed_label = smooth_segmented_labels(label)
    
        predicted_labels.append(smoothed_label)

    ###########################################################################################
    
    border_cleaned_predicted_labels = []
    
    for patch, patch_coords in zip(predicted_labels, window_coords):
        cleaned_patch = remove_border_labels(patch, patch_coords, normalized_img)
        border_cleaned_predicted_labels.append(cleaned_patch)
        
    ###########################################################################################
    
    region_info_list = compile_label_info(np.array(border_cleaned_predicted_labels), window_coords)
    
    ###########################################################################################
    
    # First, extract the bounding boxes from each region in the region_info_list
    # This creates an array of bounding boxes where each box is defined by [x_min, y_min, x_max, y_max]
    boxes = np.array([region['global_bbox'] for region in region_info_list])
    
    # Apply the Non-Maximum Suppression (NMS) function to these boxes.
    # NMS will analyze these bounding boxes and return the indices of boxes that should be kept
    # based on the overlap threshold of 0.5. Boxes that overlap more than this threshold with a larger box
    # will be filtered out.
    nms_indices = non_maximum_suppression(boxes, overlapThresh=0.5)
    
    # Using the indices obtained from NMS, construct the final list of regions.
    # This list will only include regions whose bounding boxes were selected by the NMS process,
    # effectively filtering out regions with significantly overlapping bounding boxes.
    nms_region_info_list = [region_info_list[i] for i in nms_indices]
    
    # final_region_info_list now contains the refined list of regions after applying NMS.
    # These are the regions that are considered significant based on their size and the lack of substantial
    # overlap with larger regions.
    
    ###########################################################################################
    
    canvas = place_labels_on_canvas(normalized_img, nms_region_info_list)
    
    mask_filename = os.path.join('AlgorithmPredictions', 
                                 os.path.splitext(os.path.basename(image_path))[0] + '_CellposeMask.tif')
    
    if os.path.exists(mask_filename):
        os.remove(mask_filename)
    canvas_image = Image.fromarray(canvas)
    canvas_image.save(mask_filename, format='TIFF')
    
    ###########################################################################################
    
    images_array.append(np.asarray(normalized_img))
    labels_array.append(canvas)

Reading images and predicting:   0%|          | 0/10 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

...

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

Predicting on patches:   0%|          | 0/6 [00:00<?, ?it/s]

In [11]:
modules_file = 'modules.py'
if os.path.exists(modules_file):
    os.remove(modules_file)